In [1]:
import pandas as pd
import numpy as np
from earthtext.osm.autocomplete_query import autocomplete_query

In [2]:
df = pd.read_parquet('/opt/data/california-worldcover-chips-osm-multilabels.parquet')
df.shape

(83609, 8)

In [3]:
OSM_counts = np.stack(df.onehot_count)
print(OSM_counts.shape)
print(OSM_counts)

(83609, 99)
[[  24   24 1344 ...   53    0    0]
 [ 148  148 4171 ...  166    0    1]
 [  65   65  200 ...   29    0    1]
 ...
 [   4    4    1 ...   77    0   40]
 [   0    0    0 ...    4    0   12]
 [   0    0    0 ...    0    0    7]]


In [4]:
example_constraints = np.array((0,) * 99)
example_constraints[0] = 1
example_constraints[1] = 1
example_constraints[2] = -1
example_constraints[3] = -1
example_constraints[10] = 1
example_constraints

array([ 1,  1, -1, -1,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [5]:
# %%debug
autocomplete_query(constraints=example_constraints, df=df, n_samples=20)

12


,onehot_count,onehot_area,onehot_length,string_labels,col,row,geometry,split
chip_id,,,,,,,,
29a5d117b899a,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 31, 2, 29, 0, 49, ...","[1387.1771053231764, 1387.1771053231764, 0.0, ...","[175.3093662846612, 175.3093662846612, 0.0, 0....","[amenity=*, amenity=parking, crop=*, crop=fiel...",59648,62208,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
055c53964deb1,"[5, 5, 0, 0, 0, 0, 0, 0, 0, 9, 9, 0, 0, 12, 0,...","[1588.7106419486881, 1588.7106419486881, 0.0, ...","[372.32445834181027, 372.32445834181027, 0.0, ...","[amenity=*, amenity=parking, crop=*, crop=fiel...",55296,62464,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
10c312357b680,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 12, 3, 9, 0, 11, 5...","[1936.0009179324309, 1936.0009179324309, 0.0, ...","[189.24664394866443, 189.24664394866443, 0.0, ...","[amenity=*, amenity=parking, crop=*, crop=fiel...",63744,66816,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
1e164cc44d2c8,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 6, 2, 3, 1, 102, 0...","[1480.8195388180136, 1480.8195388180136, 0.0, ...","[186.3072349785997, 186.3072349785997, 0.0, 0....","[amenity=*, amenity=parking, crop=*, crop=fiel...",66560,64512,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
35bfad9302303,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 9, 9, 0, 0, 27, 4,...","[5441.174967043069, 5441.174967043069, 0.0, 0....","[295.1525894512135, 295.1525894512135, 0.0, 0....","[amenity=*, amenity=parking, crop=*, crop=fiel...",55296,62976,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
29a5d117b899a,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 31, 2, 29, 0, 49, ...","[1387.1771053231764, 1387.1771053231764, 0.0, ...","[175.3093662846612, 175.3093662846612, 0.0, 0....","[amenity=*, amenity=parking, crop=*, crop=fiel...",59648,62208,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
065dc6317a55e,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 7, 5, 2, 0, 51, 3,...","[705.921544141705, 705.921544141705, 0.0, 0.0,...","[111.81848307312679, 111.81848307312679, 0.0, ...","[amenity=*, amenity=parking, crop=*, crop=fiel...",66816,64768,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
1c9b769f462fe,"[2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 4, 2, ...","[2136.42317929564, 2136.42317929564, 0.0, 0.0,...","[318.6417921581011, 318.6417921581011, 0.0, 0....","[amenity=*, amenity=parking, crop=*, crop=fiel...",50432,61440,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
29a5d117b899a,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 31, 2, 29, 0, 49, ...","[1387.1771053231764, 1387.1771053231764, 0.0, ...","[175.3093662846612, 175.3093662846612, 0.0, 0....","[amenity=*, amenity=parking, crop=*, crop=fiel...",59648,62208,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
